In [2]:
import torch

In [3]:
video_dir = 'datasets/mp4/'
json_dir = 'datasets/data/'
label_dir = 'datasets/cfg/'
list_file = 'datafiles.txt'

In [4]:
import numpy as np
import pandas as pd
import os
with open(list_file,'w') as datafiles:
    filelist = os.listdir(video_dir)
    for i in filelist:
        datafiles.write(i[:-4]+',')

/tmp/ipykernel_159681/928358766.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
filelist = []
with open(list_file,'r') as file:
    line = file.readline()
    filelist = line.split(',')

In [6]:
import cv2 as cv

cap = cv.VideoCapture(video_dir + filelist[0]+'.mp4')

motion = []

while cap.isOpened():
    ret, frame = cap.read()
    motion.append(frame)
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.waitKey()
cv.destroyAllWindows()


Can't receive frame (stream end?). Exiting ...


In [12]:
cv.imwrite(motion[0])